<a href="https://colab.research.google.com/github/JSJeong-me/CLIP-Zero-Shot-Classification/blob/main/Roboflow_CLIP_Zero_Shot_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use CLIP Zero-Shot on your own classificaiton dataset

This notebook provides an example of how to benchmark CLIP's zero shot classification performance on your own classification dataset.

[CLIP](https://openai.com/blog/clip/) is a new zero shot image classifier relased by OpenAI that has been trained on 400 million text/image pairs across the web. CLIP uses these learnings to make predicts based on a flexible span of possible classification categories.

CLIP is zero shot, that means **no training is required**. 

Try it out on your own task here!

Be sure to experiment with various text prompts to unlock the richness of CLIP's pretraining procedure.


![Roboflow Wordmark](https://i.imgur.com/dcLNMhV.png)


# Download and Install CLIP Dependencies

In [1]:
#installing some dependencies, CLIP was release in PyTorch
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

import numpy as np
import torch

print("Torch version:", torch.__version__)

CUDA version: 10.1
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4MB 24kB/s 
     |████████████████████████████████| 12.8MB 198kB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45613 sha256=aba640139df81eca2114e2575f85a4f25618bfce84bd3fd3c688b98cb4dd0ca2
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57fefffdbc68425b
Successfully built ftfy
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101
Torch version: 1.7.1+cu101


In [2]:
#clone the CLIP repository
!git clone https://github.com/openai/CLIP.git
%cd CLIP

Cloning into 'CLIP'...
remote: Enumerating objects: 24, done.
remote: Total 24 (delta 0), reused 0 (delta 0), pack-reused 24
Unpacking objects: 100% (24/24), done.
/content/CLIP


# Download Classification Data or Object Detection Data

We will download the [public flowers classificaiton dataset](https://public.roboflow.com/classification/flowers_classification) from Roboflow. The data will come out as folders broken into train/valid/test splits and seperate folders for each class label.

You can easily download your own dataset from Roboflow in this format, too.

We made a conversion from object detection to CLIP text prompts in Roboflow, too, if you want to try that out.


To get your data into Roboflow, follow the [Getting Started Guide](https://blog.roboflow.ai/getting-started-with-roboflow/).

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
%cd ..

/content


In [4]:
!cp /content/drive/MyDrive/Flowers_Classification.v3-augmented.clip.zip .

In [12]:
!pwd

/content


In [ ]:
#download classification data
#replace with your link
!curl -L "https://public.roboflow.com/ds/[YOUR KEY HERE]" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
!unzip ./Flowers.zip

In [14]:
import os
#our the classes and images we want to test are stored in folders in the test set
class_names = os.listdir('./test/')
class_names.remove('_tokenization.txt')
class_names

['daisy', 'dandelion']

In [15]:
#we auto generate some example tokenizations in Roboflow but you should edit this file to try out your own prompts
#CLIP gets a lot better with the right prompting!
#be sure the tokenizations are in the same order as your class_names above!
%cat ./test/_tokenization.txt

An example picture from the Flowers_Classification dataset depicting a daisy
An example picture from the Flowers_Classification dataset depicting a dandelion

In [16]:
#edit your prompts as you see fit here
%%writefile ./test/_tokenization.txt
An example picture from the flowers dataset depicting a daisy
An example picture from the flowers dataset depicting a dandelion

Overwriting ./test/_tokenization.txt


In [17]:
candidate_captions = []
with open('./test/_tokenization.txt') as f:
    candidate_captions = f.read().splitlines()

In [19]:
%cd ./CLIP/

/content/CLIP


# Run CLIP inference on your classification dataset

In [22]:
import torch
import clip
from PIL import Image
import glob

def argmax(iterable):
    return max(enumerate(iterable), key=lambda x: x[1])[0]

device = "cuda" if torch.cuda.is_available() else "cpu"
model, transform = clip.load("ViT-B/32", device=device)

correct = []

#define our target classificaitons, you can should experiment with these strings of text as you see fit, though, make sure they are in the same order as your class names above
text = clip.tokenize(candidate_captions).to(device)

for cls in class_names:
    class_correct = []
    test_imgs = glob.glob('./test/' + cls + '/*.jpg')
    for img in test_imgs:
        print(img)
        image = transform(Image.open(img)).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text)
            
            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()

            pred = class_names[argmax(list(probs)[0])]
            print(pred)
            if pred == cls:
                correct.append(1)
                class_correct.append(1)
            else:
                correct.append(0)
                class_correct.append(0)
    
    print('accuracy on class ' + cls + ' is :' + str(sum(class_correct)/len(class_correct)))
print('accuracy on all is : ' + str(sum(correct)/len(correct)))

./test/daisy/33891398493_e0b6d7f683_n_jpg.rf.056074d40fb04ac0330dcb40dc9d173d.jpg
daisy
./test/daisy/10466290366_cc72e33532_jpg.rf.9481adbf72e9d757b640219f327b2c31.jpg
dandelion
./test/daisy/14907815010_bff495449f_jpg.rf.d184c6dae55a882ebd35d816eb0f79f5.jpg
daisy
./test/daisy/6207492986_0ff91f3296_jpg.rf.f7e0e335cb1379cb92a5cea50656c528.jpg
daisy
./test/daisy/2578695910_5ab8ee17c1_n_jpg.rf.bdcce16fe2988974f32cd3fa5dec38fa.jpg
daisy
./test/daisy/2632216904_274aa17433_jpg.rf.c65a6dd63aa759e264a93d872f1985a2.jpg
daisy
./test/daisy/34571214621_f655295459_n_jpg.rf.e77f6a52632920db03e558ef5a25dc87.jpg
daisy
./test/daisy/33874126263_3f6f965784_n_jpg.rf.af5b4df8f61425ff68c02eeca5b15e05.jpg
daisy
./test/daisy/5905504340_1d60fa9611_n_jpg.rf.0a9695dc3cc728bd5e01b589d77f5313.jpg
daisy
./test/daisy/6776075110_1ea7a09dd4_n_jpg.rf.3725c92eba2068938f6a94e1d880b5d8.jpg
daisy
./test/daisy/14333681205_a07c9f1752_m_jpg.rf.431f5d547b0a428a0e6025d5acb840e0.jpg
daisy
./test/daisy/5765646947_82e95a9cc9_n_jpg.

In [ ]:
#Hope you enjoyed!
#As always, happy inferencing
#Roboflow